# Using Machine Learning to sort through proteins and peptides

Can we use protein NPX or peptide abundance to predict a patient's UPDRS 3 score? 

In [76]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [92]:
#create dataframes from csv files

peptides = pd.read_csv("PD-datasets/train_peptides.csv")
proteins = pd.read_csv("PD-datasets/train_proteins.csv")
clinical = pd.read_csv("PD-datasets/train_clinical_data.csv")

In [114]:
#cleaning and reformatting data

pep_pivot = peptides.pivot(index="visit_id",columns="Peptide",values="PeptideAbundance").sort_values(by="visit_id")
pro_pivot = proteins.pivot(index="visit_id",columns="UniProt",values="NPX")
merged_pivots = pep_pivot.merge(pro_pivot,how='left',on='visit_id')
print(merged_pivots.shape)
merged_pivots.head()

(1113, 1195)


AADDTWEPFASGK  AAFGQGSGPIMLDEVQC(UniMod_4)TGTEASLADC(UniMod_4)K  \
visit_id                                                                    
10053_0       6580710.0                                           31204.4   
10053_12      6333510.0                                           52277.6   
10053_18      7129640.0                                           61522.0   
10138_12      7404780.0                                           46107.2   
10138_24     13788300.0                                           56910.3   

          AAFTEC(UniMod_4)C(UniMod_4)QAADK  AANEVSSADVK  \
visit_id                                                  
10053_0                          7735070.0          NaN   
10053_12                         5394390.0          NaN   
10053_18                         7011920.0      35984.7   
10138_12                        10610900.0          NaN   
10138_24                         6906160.0      13785.5   

          AATGEC(UniMod_4)TATVGKR  AATVGSLAGQPLQER  AAVYHHFISDGVR  \
visit_id                                                            
10053_0                       NaN              NaN        46620.3   
10053_12                      NaN              NaN        57554.5   
10053_18                  17188.0          19787.3        36029.4   
10138_12                  20910.2          66662.3        55253.9   
10138_24                  11004.2          63672.7        36819.8   

          ADDKETC(UniMod_4)FAEEGK  ADDKETC(UniMod_4)FAEEGKK  \
visit_id                                                      
10053_0                  236144.0                       NaN   
10053_12                 108298.0                   45885.4   
10053_18                 708729.0                 5067790.0   
10138_12                  79575.5                 6201210.0   
10138_24                  34160.9                 2117430.0   

          ADDLGKGGNEESTKTGNAGSR  ...    Q9HDC9    Q9NQ79    Q9NYU2   Q9UBR2  \
visit_id                         ...                                          
10053_0                     NaN  ...       NaN   9469.45   94237.6      NaN   
10053_12                    NaN  ...       NaN  14408.40       NaN      NaN   
10053_18                30838.2  ...  317477.0  38667.20  111107.0      NaN   
10138_12                26720.0  ...  557904.0  44556.90  155619.0  14647.9   
10138_24                15645.2  ...       NaN  47836.70  177619.0  17061.1   

           Q9UBX5    Q9UHG2    Q9UKV8   Q9UNU6    Q9Y646    Q9Y6R7  
visit_id                                                            
10053_0   23016.0  177983.0   65900.0  15382.0       NaN  19017.40  
10053_12  28537.0  171733.0   65668.1      NaN   9295.65  25697.80  
10053_18  37932.6  245188.0   59986.1  10813.3       NaN  29102.70  
10138_12  36927.7  229232.0  106564.0  26077.7  21441.80   7642.42  
10138_24  25510.4  176722.0   59471.4  12639.2  15091.40   6168.55  

[5 rows x 1195 columns]

In [96]:
# completing the data set with patient context

full_merge = merged_pivots.merge(clinical,how='left',on='visit_id')
print(full_merge.shape)
full_merge.head()

(1113, 1203)


visit_id  AADDTWEPFASGK  AAFGQGSGPIMLDEVQC(UniMod_4)TGTEASLADC(UniMod_4)K  \
0   10053_0      6580710.0                                           31204.4   
1  10053_12      6333510.0                                           52277.6   
2  10053_18      7129640.0                                           61522.0   
3  10138_12      7404780.0                                           46107.2   
4  10138_24     13788300.0                                           56910.3   

   AAFTEC(UniMod_4)C(UniMod_4)QAADK  AANEVSSADVK  AATGEC(UniMod_4)TATVGKR  \
0                         7735070.0          NaN                      NaN   
1                         5394390.0          NaN                      NaN   
2                         7011920.0      35984.7                  17188.0   
3                        10610900.0          NaN                  20910.2   
4                         6906160.0      13785.5                  11004.2   

   AATVGSLAGQPLQER  AAVYHHFISDGVR  ADDKETC(UniMod_4)FAEEGK  \
0              NaN        46620.3                 236144.0   
1              NaN        57554.5                 108298.0   
2          19787.3        36029.4                 708729.0   
3          66662.3        55253.9                  79575.5   
4          63672.7        36819.8                  34160.9   

   ADDKETC(UniMod_4)FAEEGKK  ...   Q9UNU6    Q9Y646    Q9Y6R7  patient_id  \
0                       NaN  ...  15382.0       NaN  19017.40     10053.0   
1                   45885.4  ...      NaN   9295.65  25697.80     10053.0   
2                 5067790.0  ...  10813.3       NaN  29102.70     10053.0   
3                 6201210.0  ...  26077.7  21441.80   7642.42     10138.0   
4                 2117430.0  ...  12639.2  15091.40   6168.55     10138.0   

   visit_month  updrs_1  updrs_2  updrs_3  updrs_4  \
0          0.0      3.0      0.0     13.0      0.0   
1         12.0      4.0      2.0      8.0      0.0   
2         18.0      2.0      2.0      0.0      0.0   
3         12.0      3.0      6.0     31.0      0.0   
4         24.0      4.0      7.0     19.0     10.0   

   upd23b_clinical_state_on_medication  
0                                  NaN  
1                                  NaN  
2                                  NaN  
3                                   On  
4                                   On  

[5 rows x 1203 columns]

In [97]:
# using label encoding to prepare the data for ML

le = LabelEncoder()
encoded_mm = full_merge[["upd23b_clinical_state_on_medication"]].apply(le.fit_transform)
full_merge["upd23b_clinical_state_on_medication"] = encoded_mm["upd23b_clinical_state_on_medication"]
full_merge.head()

visit_id  AADDTWEPFASGK  AAFGQGSGPIMLDEVQC(UniMod_4)TGTEASLADC(UniMod_4)K  \
0   10053_0      6580710.0                                           31204.4   
1  10053_12      6333510.0                                           52277.6   
2  10053_18      7129640.0                                           61522.0   
3  10138_12      7404780.0                                           46107.2   
4  10138_24     13788300.0                                           56910.3   

   AAFTEC(UniMod_4)C(UniMod_4)QAADK  AANEVSSADVK  AATGEC(UniMod_4)TATVGKR  \
0                         7735070.0          NaN                      NaN   
1                         5394390.0          NaN                      NaN   
2                         7011920.0      35984.7                  17188.0   
3                        10610900.0          NaN                  20910.2   
4                         6906160.0      13785.5                  11004.2   

   AATVGSLAGQPLQER  AAVYHHFISDGVR  ADDKETC(UniMod_4)FAEEGK  \
0              NaN        46620.3                 236144.0   
1              NaN        57554.5                 108298.0   
2          19787.3        36029.4                 708729.0   
3          66662.3        55253.9                  79575.5   
4          63672.7        36819.8                  34160.9   

   ADDKETC(UniMod_4)FAEEGKK  ...   Q9UNU6    Q9Y646    Q9Y6R7  patient_id  \
0                       NaN  ...  15382.0       NaN  19017.40     10053.0   
1                   45885.4  ...      NaN   9295.65  25697.80     10053.0   
2                 5067790.0  ...  10813.3       NaN  29102.70     10053.0   
3                 6201210.0  ...  26077.7  21441.80   7642.42     10138.0   
4                 2117430.0  ...  12639.2  15091.40   6168.55     10138.0   

   visit_month  updrs_1  updrs_2  updrs_3  updrs_4  \
0          0.0      3.0      0.0     13.0      0.0   
1         12.0      4.0      2.0      8.0      0.0   
2         18.0      2.0      2.0      0.0      0.0   
3         12.0      3.0      6.0     31.0      0.0   
4         24.0      4.0      7.0     19.0     10.0   

   upd23b_clinical_state_on_medication  
0                                    2  
1                                    2  
2                                    2  
3                                    1  
4                                    1  

[5 rows x 1203 columns]

In [98]:
# final steps to prep for ML - making sure we have no NAN


clean_df = full_merge[full_merge.upd23b_clinical_state_on_medication != 2]
clean_df = clean_df.fillna(0)
clean_df.head()

visit_id  AADDTWEPFASGK  AAFGQGSGPIMLDEVQC(UniMod_4)TGTEASLADC(UniMod_4)K  \
3   10138_12      7404780.0                                           46107.2   
4   10138_24     13788300.0                                           56910.3   
5   10138_36      6924040.0                                           64313.5   
13  10541_60      5578010.0                                           61435.3   
14  10541_84      4339140.0                                           18940.0   

    AAFTEC(UniMod_4)C(UniMod_4)QAADK  AANEVSSADVK  AATGEC(UniMod_4)TATVGKR  \
3                         10610900.0          0.0                 20910.20   
4                          6906160.0      13785.5                 11004.20   
5                          9098610.0      26400.9                 18784.60   
13                         4902210.0          0.0                 11377.50   
14                         4370350.0      26725.7                  8668.14   

    AATVGSLAGQPLQER  AAVYHHFISDGVR  ADDKETC(UniMod_4)FAEEGK  \
3           66662.3        55253.9                  79575.5   
4           63672.7        36819.8                  34160.9   
5           78724.1        50155.8                 508579.0   
13         107775.0        40710.1                 330576.0   
14          94758.2        36451.5                 325450.0   

    ADDKETC(UniMod_4)FAEEGKK  ...   Q9UNU6   Q9Y646    Q9Y6R7  patient_id  \
3                  6201210.0  ...  26077.7  21441.8   7642.42     10138.0   
4                  2117430.0  ...  12639.2  15091.4   6168.55     10138.0   
5                  5601500.0  ...      0.0  22910.5   9596.48     10138.0   
13                 2868310.0  ...      0.0  23121.7  17482.50     10541.0   
14                 2418020.0  ...      0.0  16998.2  16885.90     10541.0   

    visit_month  updrs_1  updrs_2  updrs_3  updrs_4  \
3          12.0      3.0      6.0     31.0      0.0   
4          24.0      4.0      7.0     19.0     10.0   
5          36.0      5.0      2.0     11.0      0.0   
13         60.0      5.0      3.0     29.0      3.0   
14         84.0      5.0      9.0     38.0      3.0   

    upd23b_clinical_state_on_medication  
3                                     1  
4                                     1  
5                                     1  
13                                    0  
14                                    0  

[5 rows x 1203 columns]

In [115]:
# transforming our target into a binary, using what we know about the UPDRS scale

clean_df['updrs_3'].mask(clean_df['updrs_3'] >= 22 ,1, inplace=True)
clean_df['updrs_3'].mask(clean_df['updrs_3'] != 1 ,0, inplace=True)

Part I: Nonmotor experiences of daily living: 13 items. Score range: 0–52,[8] 10 and below is mild, 22 and above is severe.[9]
Part II: Motor experiences of daily living: 13 items. Score range: 0–52,[8] 12 and below is mild, 30 and above is severe.[9]
Part III: Motor examination: 18 items. Score range: 0–132,[8] 32 and below is mild, 59 and above is severe.[9]
Part IV: Motor complications: 6 items. Score range: 0–24,[8] 4 and below is mild, 13 and above is severe.[9]
https://en.wikipedia.org/wiki/Unified_Parkinson%27s_disease_rating_scale

In [116]:
clean_df['updrs_3'].value_counts()

1.0    263
0.0    189
Name: updrs_3, dtype: int64

In [101]:
OUTCOME_COLUMNS = ['updrs_1', 'updrs_2', 'updrs_3', 'updrs_4','patient_id']

In [102]:
clean_df = clean_df.set_index("visit_id")

In [103]:
y = clean_df['updrs_3'].values.reshape(-1,1)
X = clean_df.drop(columns=OUTCOME_COLUMNS)

In [104]:
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [105]:
# Create the StandardScaler instance
scaler = StandardScaler()

In [106]:
# Fit the Standard Scaler with the training data
X_scaler = scaler.fit(X_train)

In [107]:
# Scale the training data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [108]:
# Create the random forest classifier instance
rf_model = RandomForestClassifier(n_estimators=100, random_state=78)

In [109]:
# Fit the model and use .ravel()on the "y_train" data. 
rf_model = rf_model.fit(X_train_scaled, y_train.ravel())

In [110]:
# Making predictions using the testing data
predictions = rf_model.predict(X_test_scaled)

In [111]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)

In [112]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


Predicted 0  Predicted 1
Actual 0           21           29
Actual 1           14           49

Accuracy Score : 0.6194690265486725
Classification Report
              precision    recall  f1-score   support

         0.0       0.60      0.42      0.49        50
         1.0       0.63      0.78      0.70        63

    accuracy                           0.62       113
   macro avg       0.61      0.60      0.59       113
weighted avg       0.62      0.62      0.61       113



In [113]:
# Get the feature importance array
importances = rf_model.feature_importances_
# List the top 10 most important features
importances_sorted = sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)
importances_sorted[:10]

[(0.007315921394847524, 'EALQGVGDMGR'),
 (0.00585122184424807, 'QGVNDNEEGFFSAR'),
 (0.005469324111095433, 'TLKIENVSYQDKGNYR'),
 (0.005058058160248663, 'upd23b_clinical_state_on_medication'),
 (0.004297572853784166, 'NFPPSQDASGDLYTTSSQLTLPATQC(UniMod_4)LAGK'),
 (0.004256820647213172, 'Q13283'),
 (0.004157691524294517, 'FTFEYSR'),
 (0.0037879539346434787,
  'C(UniMod_4)QC(UniMod_4)DELC(UniMod_4)SYYQSC(UniMod_4)C(UniMod_4)TDYTAEC(UniMod_4)KPQVTR'),
 (0.0037361400602436657, 'TFTC(UniMod_4)TAAYPESK'),
 (0.0036761875068712802, 'P35542')]

# Interpreting our analysis

While we were not able to accurately predict UPDRS scores with the Random Forest model, we did get some valuable insight from the feature importance - we can investigate the peptides and proteins listed here as next steps in better understanding biomarkers for Parkinson's Progression.